In [1]:
%load_ext autoreload
%autoreload 2

In [9]:
%env PYTHONPATH = /mnt/nvme0n1p3/kaggle_humpback_new_whale
%env PROJECT_PATH = /mnt/nvme0n1p3/kaggle_humpback_new_whale
%cd /mnt/nvme0n1p3/kaggle_humpback_new_whale

env: PYTHONPATH=/mnt/nvme0n1p3/kaggle_humpback_new_whale
env: PROJECT_PATH=/mnt/nvme0n1p3/kaggle_humpback_new_whale
/mnt/nvme0n1p3/kaggle_humpback_new_whale


In [38]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import os
from sklearn.externals import joblib
from utils.envs import *
from utils.config import *
from utils.common import get_label, remove_new_whale, create_label
from model.arch import se_resnext101
from model.dataset import train_transform, test_transform, TestDataset, TrainDataset
from sklearn.model_selection import train_test_split

In [39]:
train_df = pd.read_csv(train_repo)
train_df = remove_new_whale(train_df)
dev_df, val_df = train_test_split(train_df, test_size = 0.20)

In [40]:
ohe_model, train_image_label, train_ohe_label = create_label(train_df)
joblib.dump(ohe_model, os.path.join(artifacts_path, 'ohe_model.pkl'))

['/home/aditya/extrahd/kaggle_humpback_new_whale/output/artifacts/ohe_model.pkl']

In [44]:
dev_image_label, dev_ohe_label = get_label(dev_df, ohe_model)
val_image_label, val_ohe_label = get_label(val_df, ohe_model)
test_image_label = [x for x in os.listdir(test_path) if x.endswith('.jpg')]

In [47]:
dev_dataset = TrainDataset(dev_image_label, dev_ohe_label, train_path, train_transform, DEVICE)
val_dataset = TrainDataset(val_image_label, train_path, test_transform, DEVICE)
test_dataset = TestDataset(test_image_label, test_path, test_transform, DEVICE)

In [19]:
dev_loader = DataLoader(dev_dataset, batch_size = 32, shuffle = True, num_workers = 0)
val_loader = DataLoader(val_dataset, batch_size = 32, shuffle = False, num_workers = 0)
test_loader = DataLoader(test_dataset, batch_size = 32, shuffle = False, num_workers = 0)

In [24]:
model = se_resnext101(n_classes=N_CLASSES, pretrained = True)
n_epoch = 10
dev_dataloader = train_loader
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0005)
scheduler = lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)

Downloading: "http://data.lip6.fr/cadene/pretrainedmodels/se_resnext101_32x4d-3b2fe3d8.pth" to /home/aditya/.torch/models/se_resnext101_32x4d-3b2fe3d8.pth
100%|██████████| 196466866/196466866 [00:18<00:00, 10640329.05it/s]
